In [1]:
#Python API for ltr11 is required
#from ltr11 import ∗
import ltr11
import threading
import concurrent.futures
import time
import pprint
import queue
import csv
import serial
import os
import numpy as np
import re
import cv2
import pyrealsense2 as rs

In [2]:
def get_rPPG_data (event, queue) :
    # Configure depth and color streams
    flag = 0
    pipeline = rs.pipeline()
    config = rs.config()

    # Get device product line for setting a supporting resolution
    pipeline_wrapper = rs.pipeline_wrapper(pipeline)
    pipeline_profile = config.resolve(pipeline_wrapper)
    device = pipeline_profile.get_device()
    device_product_line = str(device.get_info(rs.camera_info.product_line))

    found_rgb = False
    for s in device.sensors:
        if s.get_info(rs.camera_info.name) == 'RGB Camera':
            found_rgb = True
            break
    if not found_rgb:
        print("The demo requires Depth camera with Color sensor")
        exit(0)

    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

    if device_product_line == 'L500':
        config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 30)
    else:
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
    profile = pipeline.start(config)
    # Getting the depth sensor's depth scale (see rs-align example for explanation)
    depth_sensor = profile.get_device().first_depth_sensor()
    depth_scale = depth_sensor.get_depth_scale()
    print("Depth Scale is: " , depth_scale)

    s=0
    time_sequence = []
    frame = 0
    time_str = time.strftime("%Y%m%d-%H%M%S")
    writer = cv2.VideoWriter('rPPG original {}.mp4'.format(time_str), -1, 19, (640, 480))
    writer1 = cv2.VideoWriter('rPPG depth {}.mp4'.format(time_str), -1, 19, (640, 480))

    # Create an align object
    # rs.align allows us to perform alignment of depth frames to others frames
    # The "align_to" is the stream type to which we plan to align depth frames.
    align_to = rs.stream.color
    align = rs.align(align_to)

    start_time = time.time()

    while not event.is_set():
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames() 
        # Align the depth frame to color frame
        aligned_frames = align.process(frames)
        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        if not aligned_depth_frame or not color_frame:
            print('no cohorent frames')
            continue
        frame_time = time.time()-start_time
        time_sequence.append(frame_time)
        frame = frame+1
        # Convert images to numpy arrays
        depth = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = cv2.convertScaleAbs(depth, alpha=255/2000)
        
        if flag == 0:
            print('first rPPG data at {}'.format(time.time()))
            flag = 1

        #save the video
        writer.write(color_image)
        writer1.write(depth_image)
        #writer1.write(depth_image)

        # Display
        cv2.imshow('img', color_image)
        cv2.setWindowProperty('img', cv2.WND_PROP_TOPMOST, 1)

        # Stop if escape key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            pipeline.stop()
            break
    cv2. destroyAllWindows()
    # Release the VideoCapture object
    writer.release()
    writer1.release()
    run_time = time.time()-start_time
    fps = frame/run_time

    fname = "{}-{}.csv".format('rPPG original time sequence',time_str)
    fpath = "{}".format(fname)
    f = open(fpath, 'w', newline="")
    csv.writer(f).writerows(map(lambda x: [x], time_sequence))
    f.close()
    print(('--- %s seconds ---' %(run_time)))
    print(('--- %s fps ---' %(fps)))

In [3]:
def get_ecg_data(event, serial, queue):
    flag = True
    try:
        serial.close()
        serial.open()
        while not event.is_set():
            data = []
            for i in range(250):    # 500 Hz / 2
                line = serial.readline()
                #there are four values in the line_sequence = [-ECG1 -ECG2 -ECG3 -temperature sensor]
                line_sequence = line.decode("utf-8")
                #print('This is the line_sequence from the serial port:{}'.format(line_sequence))
                #use match function to record the ECG1
                #match = re.findall(r'\-?\d+', line_sequence)
                #data += [int(match[0])]
                data += [int(line_sequence)]
                #print('This is the ECG value selected from the serial port:{}'.format(int(match[0])))
                if flag:
                    print('first ecg data at {}'.format(time.time()))
                    flag = False
            for row in zip(data):
                queue.put(row)
        print('ecg stopped {}\n'.format(time.time()))
        serial.close()
        print('serial port closed')
        queue.put([])  #Prevent queue.get()  call getting stuck
    except:
        print('ecg error')
        serial.close()
        print('serial port closed')
        queue.put([])  #Prevent queue.get()  call getting stuck
        event.set()

In [4]:
def write_data(event, queue, f):
    try:
        wr = csv.writer(f)
        while not (event.is_set() and queue.empty()):
            #if event.is_set():
                #print('event is set')
                #if not queue.empty():
                    #print('queue is still not empty!')
            data = queue.get()
            wr.writerow(data)
        print('Closing {}'.format(os.path.basename(f.name)))
        f.close()
        print('{} closed'.format(os.path.basename(f.name)))
    except:
        print('{} writer error'.format(f))

In [5]:
def init():
    print('init')
    ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1, 1)
    return ln,

In [6]:
def update(frame):
    print('update')
    xdata.append(frame)
    ydata.append(np.sin(frame))
    ln.set_data(xdata, ydata)
    return ln,

In [7]:
if __name__ == "__main__":

    # To capture video from webcam. 
    #cap = cv2.VideoCapture(0)

    sample_dur = 40  # Sample duration in seconds
    start = time.time()
    timestr = time.strftime("%Y%m%d-%H%M%S")
    fname = "{}-{}.csv".format(timestr,'ecg')
    fpath = "{}".format(fname)
    f2 = open(fpath, 'w', newline="")
    
    event = threading.Event()
    #radar_pipeline = queue.Queue()
    ecg_pipeline = queue.Queue()
    rPPG_pipeline = queue.Queue()
    
    # Set up serial linstener
    #serial_port = 'COM5'
    #the serial_port used in my laptop is actually COM3 instead of COM5
    serial_port = 'COM3'
    #baud_rate = 250000
    baud_rate = 921600
    ser = serial.Serial(serial_port, baud_rate)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        print('starting threads')
        rPPG_thread = executor.submit(get_rPPG_data, event, rPPG_pipeline)
        ecg_thread = executor.submit(get_ecg_data, event, ser, ecg_pipeline)
        #radar_writer = executor.submit(write_data, event, radar_pipeline, f1)
        ecg_writer = executor.submit(write_data, event, ecg_pipeline, f2)
        try:
            while not (ecg_thread.done() and ecg_writer.done() and rPPG_thread.done()):
                if time.time() - start >= sample_dur:
                    event.set()
                else:
                    time.sleep(0.5)
                    
        except KeyboardInterrupt:
            print('Keyboard Interrupt')
            event.set()
            
        #print('Radar result: {}'.format(radar_thread.result()))
        print('Radar result: {}'.format(ecg_thread.result()))
        #print('Radar writer result:{}'.format(radar_writer.result()))
        print('ECG writer result:{}'.format(ecg_writer.result()))
        print('elapsed time = {}'.format(time.time() - start))
        
        
        

starting threads
Depth Scale is:  0.0010000000474974513
first rPPG data at 1684401898.3801162
first ecg data at 1684401899.175807
--- 39.06999659538269 seconds ---
--- 28.333762387141487 fps ---
ecg stopped 1684401956.3227534

serial port closed
Closing 20230518-102456-ecg.csv
20230518-102456-ecg.csv closed
Radar result: None
ECG writer result:None
elapsed time = 59.538912534713745


In [8]:
1684232138.6692643-1684232137.8782585

0.7910058498382568

In [9]:
1684233955.6374514-1684233954.851871

0.7855803966522217

In [11]:
1899.175807-1898.3801162

0.7956908000001022